In [ ]:
#@title Imports

#!pip install Unidecode
#from unidecode import unidecode
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
import ast
from matplotlib.lines import Line2D

from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import scipy.stats

from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
import numpy as np
import random

all_the_years = np.arange(1974, 2023, 1)
some_of_the_years = np.arange(1997, 2023, 1)
pd.set_option('display.colheader_justify', 'center')


sns.set_style('darkgrid')        # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

import re

In [ ]:
#@title Import Selenium
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
print_single_player_nba_shooting_data('Damian Lillard', f"https://www.basketball-reference.com/players/l/lillada01.html")

In [ ]:
#@title print_single_player_nba_shooting_data function

def print_single_player_nba_shooting_data(player, player_url):
 
    final_player_shooting_df = pd.DataFrame(columns = ['Lg', 'G', 'MP', 'FG%', 'Dist.', 
                                                          '2Pfreq', '0-3freq', '3-10freq', '10-16freq', '16-3Pfreq', '3Pfreq', '2P', '0-3', '3-10', 
                                                          '10-16', '16-3P', '3P', '2P', '3P',	'%FGAdunks',	'#dunks',	'%3PAcorner',
                                                          '3P%corner', 'Heave Att.', 'Heave Mad.'
                                                         ])

    wd = webdriver.Chrome('chromedriver',options=chrome_options)
    wd.get(player_url)          
    html = wd.page_source
    soup = BeautifulSoup(html)

    for first_div in soup.find_all('div', attrs={'id': 'all_shooting-playoffs_shooting'}):
      second_div = first_div.find('div', attrs={'id': 'switcher_shooting-playoffs_shooting'})
      first_table = second_div.find('table', attrs={'id': 'playoffs_shooting'})
      foot = first_table.find('tfoot')

    rows = foot.find('tr')
    player_shooting_distribution_stats = [[td.getText() for td in rows.findAll('td')]]

    # remove empty rows
    player_shooting_distribution_stats = [e for e in player_shooting_distribution_stats if e != []]

    new_list = [[s for s in sub_list if s] for sub_list in player_shooting_distribution_stats]

    headers = ['Lg', 'G', 'MP', 'FG%', 'Dist.', 
               '2Pfreq', '0-3freq', '3-10freq', '10-16freq', '16-3Pfreq', '3Pfreq', '2P', '0-3', '3-10', 
               '10-16', '16-3P', '3P', '2P', '3P',	'%FGAdunks',	'#dunks',	'%3PAcorner',
               '3P%corner', 'Heave Att.', 'Heave Mad.']

    each_year = pd.DataFrame(new_list, columns = headers)

    # append dataframe
    final_player_shooting_df = final_player_shooting_df.append(each_year)

    new_df = pd.DataFrame(columns = ['Player', 'Midrange PPS', 'Mid Freq', 'PP75', 'TS+', 'MP'])
    headers_list = ['Player', 'Midrange PPS', 'Mid Freq', 'PP75', 'TS+', 'MP']

    freq = float(final_player_shooting_df['10-16freq']) + float(final_player_shooting_df['16-3Pfreq'])


    pps = ((float(final_player_shooting_df['10-16freq']) / freq) * ( ( float(final_player_shooting_df['10-16'] )*2 ))) + ((float(final_player_shooting_df['16-3Pfreq']) / freq) * ( ( float(final_player_shooting_df['16-3P'] ) *2)))
    round(pps, 3)
    round(freq, 3)

    (player, midrange_PPS, mid_Freq, pp75, ts, mp) = playoff_career_scoring_output(player, pps, freq, import_player_since74playoffs_per75_df)
    new_df.loc[len(new_df)] = [player, midrange_PPS, mid_Freq, pp75, ts, mp]
    outfile = f"{player}_Playoff_Midrange_Data.csv"
    print(new_df)
    new_df.to_csv(outfile, index=False)
def playoff_career_scoring_output(player, pps, freq, postseason_data):
  player_data = postseason_data[(postseason_data.Player == player)]

  total_mp = 0
  total_pts = 0
  total_ts = 0
  mp_list = []

  # find total minutes a
  for row in player_data['MP']:
    mp_list.append(row)
    total_mp += row
  if total_mp == 0:
    total_mp = 1

  # find total PTS a
  i = 0
  for row in player_data['PTS']:
    total_pts += row * (mp_list[i] / total_mp)
    i = i + 1

  # find total TS+ a
  i = 0
  for row in player_data['TS%+']:
    total_ts += row * (mp_list[i] / total_mp)
    i = i + 1

  round(total_pts, 2)
  total_pts = '%.2f' % round(total_pts, 2)
  total_ts = '%.2f' % round(total_ts, 2)
  pps = '%.3f' % round(pps, 3)
  return (player, pps, freq, total_pts, total_ts, total_mp)

In [ ]:
#@title nba playoff ALL shooting zone scrapes

def nba_all_shooting_data(url_df):

    new_df = pd.DataFrame(columns = ['Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'PP75', 'TS+', 'MP'])
 
    for a, b in url_df.itertuples(index=False):
      player = a
      url = b
      url = "https://www.basketball-reference.com" + url
      

      wd = webdriver.Chrome('chromedriver',options=chrome_options)
      wd.get(url)          
      html = wd.page_source
      soup = BeautifulSoup(html)




      # scrape Playoff shooting distribution data
      for first_div in soup.find_all('div', attrs={'id': 'all_shooting-playoffs_shooting'}):
        second_div = first_div.find('div', attrs={'id': 'switcher_shooting-playoffs_shooting'})
        first_table = second_div.find('table', attrs={'id': 'playoffs_shooting'})
        foot = first_table.find('tfoot')

      rows = foot.find('tr')
      player_shooting_distribution_stats = [[td.getText() for td in rows.findAll('td')]]

      # remove empty rows
      player_shooting_distribution_stats = [e for e in player_shooting_distribution_stats if e != []]

      headers = ['dum1', 'dum2', 'Lg', 'dum3', 'G', 'MP', 'FG%', 'Dist.', 'dum4',
                '2Pfreq', '0-3freq', '3-10freq', '10-16freq', '16-3Pfreq', '3Pfreq', 
                 'dum5', '2P', '0-3', '3-10', '10-16', '16-3P', '3P', 'dum6', '2Pass', 
                 '3Pass', 'dum7', '%FGAdunks',	'#dunks',	'dum8', '%3PAcorner', '3P%corner', 'dum9', 'Heave Att.', 'Heave Mad.']

      shooting_dist_data = pd.DataFrame(player_shooting_distribution_stats, columns = headers)

      # drop empty cells
      shooting_dist_data = shooting_dist_data.drop(['dum1', 'dum2', 'dum3', 'dum4', 'dum5', 'dum6', 'dum7', 'dum8', 'dum9'], axis=1)

      tmp_df = pd.DataFrame(columns = ['Player', '0-3 feet PPS', '0-3 Freq', '0-3 FGA per 75', '3-10 feet PPS', '3-10 Freq', '3-10 FGA per 75', '10-16 feet PPS', '10-16 Freq', '10-16 FGA per 75', '16-3P feet PPS', '16-3P Freq', '16-3P FGA per 75', '3P PPS', '3P Freq', '3P FGA per 75', 'PP75', 'TS+', 'MP'])

      if int(shooting_dist_data['MP']) <= 1000:
        continue
      
      freq = [0, 0, 0, 0, 0]
      pps = [0, 0, 0, 0, 0]
      no_long2 = 0
      no_short2 = 0
      no_3 = 0
      freq_0 = float(shooting_dist_data['0-3freq'][0])
      freq_1 = float(shooting_dist_data['3-10freq'][0])
      if shooting_dist_data['10-16freq'][0] == '.000':
        no_short2 = 1
      freq_2 = float(shooting_dist_data['10-16freq'][0])
      if shooting_dist_data['16-3Pfreq'][0] == '.000':
        no_long2 = 1
      freq_3 = float(shooting_dist_data['16-3Pfreq'][0])
      if shooting_dist_data['3Pfreq'][0] == '.000':
        no_3 = 1
      freq_4 = float(shooting_dist_data['3Pfreq'][0])
      round(freq_0, 3)
      round(freq_1, 3)
      round(freq_2, 3)
      round(freq_3, 3)
      round(freq_4, 3)

      pps_0 = float(shooting_dist_data['0-3'])*2
      pps_1 = float(shooting_dist_data['3-10'])*2
      if no_short2 == 1:
        pps_2 = 0.00
      else:
        pps_2 = float(shooting_dist_data['10-16'])*2
      if no_long2 == 1:
        pps_3 = 0.00
      else:
        pps_3 = float(shooting_dist_data['16-3P'])*2
      if no_3 == 1:
        pps_4 = 0.00
      else:
        pps_4 = float(shooting_dist_data['3P'])*2
      round(pps_0, 3)
      round(pps_1, 3)
      round(pps_2, 3)
      round(pps_3, 3)
      round(pps_4, 3)
      




      # scrape Playoff FGA per 100 possessions
      for first_div in soup.find_all('div', attrs={'id': 'all_per_poss-playoffs_per_poss'}):
        second_div = first_div.find('div', attrs={'id': 'switcher_per_poss-playoffs_per_poss'})
        first_table = second_div.find('table', attrs={'id': 'playoffs_per_poss'})
        foot = first_table.find('tfoot')

      rows = foot.find('tr')
      player_per_100_data = [[td.getText() for td in rows.findAll('td')]]

      # remove empty rows
      player_per_100_data = [e for e in player_per_100_data if e != []]

      headers = ['Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 
                 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%','FT', 'FTA', 'FT%', 
                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'dum1',
                 'ORtg', 'DRtg']

      playoff_fga = pd.DataFrame(player_per_100_data, columns = headers)

      # append dataframe
      playoff_fga = playoff_fga.drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 
                 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%','FT', 'FTA', 'FT%', 
                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'dum1',
                 'ORtg', 'DRtg'], axis=1)
      playoff_fga_per_75_0 = (float(playoff_fga['FGA']) * .75) * freq_0
      playoff_fga_per_75_1 = (float(playoff_fga['FGA']) * .75) * freq_1
      playoff_fga_per_75_2 = (float(playoff_fga['FGA']) * .75) * freq_2
      playoff_fga_per_75_3 = (float(playoff_fga['FGA']) * .75) * freq_3
      playoff_fga_per_75_4 = (float(playoff_fga['FGA']) * .75) * freq_4
      playoff_fga_per_75_0 = '%.2f' % round(playoff_fga_per_75_0, 2)
      playoff_fga_per_75_1 = '%.2f' % round(playoff_fga_per_75_1, 2)
      playoff_fga_per_75_2 = '%.2f' % round(playoff_fga_per_75_2, 2)
      playoff_fga_per_75_3 = '%.2f' % round(playoff_fga_per_75_3, 2)
      playoff_fga_per_75_4 = '%.2f' % round(playoff_fga_per_75_4, 2)


      (pp75, ts, mp) = playoff_career_scoring_output(player, era_opponent_adj_playoff_per_75_df)
      tmp_df.loc[len(tmp_df)] = [player, pps_0, freq_0, playoff_fga_per_75_0, pps_1, freq_1, playoff_fga_per_75_1, pps_2, freq_2, playoff_fga_per_75_2, pps_3, freq_3, playoff_fga_per_75_3,pps_4, freq_4, playoff_fga_per_75_4, pp75, ts, mp]
      new_df = new_df.append(tmp_df)
      print(tmp_df)
      outfile = f"NBA_Playoff_all_shooting_data_df.csv"
      new_df.to_csv(outfile, index=False)
def playoff_career_scoring_output(player, postseason_data):
  player_data = postseason_data[(postseason_data.Player == player)]
  player_data = player_data[(player_data.Year >= 1997)]

  total_mp = 0
  total_pts = 0
  total_ts = 0
  mp_list = []

  # find total minutes a
  for row in player_data['MP']:
    mp_list.append(row)
    total_mp += row
  if total_mp == 0:
    total_mp = 1

  # find total PTS a
  i = 0
  for row in player_data['PTS']:
    total_pts += row * (mp_list[i] / total_mp)
    i = i + 1

  # find total TS+ a
  i = 0
  for row in player_data['TS%+']:
    total_ts += row * (mp_list[i] / total_mp)
    i = i + 1

  round(total_pts, 2)
  total_pts = '%.2f' % round(total_pts, 2)
  total_ts = '%.2f' % round(total_ts, 2)
  return (total_pts, total_ts, total_mp)

In [ ]:
player_urls = pd.read_csv('/content/nba_player_since97_URL_data_1000_min.csv', index_col=False, encoding='utf8')
player_urls = player_urls.drop(columns=['MP'])
nba_all_shooting_data(player_urls)

**PLOT SHOOTING DISTRIBUTION DATA**

In [ ]:
#@title Midrange Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['Mid FGA per 75'] = graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
total_freq = graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['Midrange PPS'] = (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])
graph_data = graph_data.dropna()
graph_data = graph_data[(graph_data['Mid FGA per 75'] >= 2)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tony Parker')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Manu Ginóbili'),'#FF8200', graph_data.color)

size = 100 * ((graph_data['MP'] - graph_data['MP'].min()) / (graph_data['MP'].max() - graph_data['MP'].min()))
plt.figure(figsize=(17,9), tight_layout=True)
plt.ylim(0.4, 1.3)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="Mid FGA per 75", y="Midrange PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='Playoff Midrange (10-3P) Shooting (1997-2022) >800 MP', xlabel='Mid FGA per 75', ylabel='Mid PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tony Parker', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Manu Ginóbili', markerfacecolor='#FF8200',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')


# ridge regression
x_pred = graph_data['Mid FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['Midrange PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
#ax.plot(x_pred, pred, color="", linewidth=3)
selected_dots = graph_data[(graph_data['Player'] != 'Tony Parker') & (graph_data['Player'] != 'Manu Ginóbili') & ((graph_data['Player'] == 'John Stockton') | (graph_data['Player'] == 'Seth Curry') 
| (graph_data['Player'] == 'Chris Paul') | (graph_data['Player'] == 'Michael Jordan') | (graph_data['Player'] == 'Kevin Durant')
| (graph_data['Player'] == 'Dirk Nowitzki') | (graph_data['Player'] == 'Stephen Curry') | (graph_data['Player'] == 'Kobe Bryant')
| (graph_data['Player'] == 'Damian Lillard') | (graph_data['Player'] == 'Dennis Rodman')
| (graph_data['Player'] == 'DeMarre Carroll') | (graph_data['MP'] >= 6000))]
label_point(selected_dots['Mid FGA per 75'], selected_dots['Midrange PPS'], selected_dots['Player'], plt.gca())
plt.savefig(f'Midrange_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-3 Graph
graph_data =  pd.read_csv('/content/nba_player_0-3_playoff_shooting_data.csv', encoding='utf8')
graph_data = graph_data[(graph_data['0-3 FGA per 75'] >= 4)]
graph_data = graph_data.sort_values('Player', ascending=False)

graph_data = graph_data[(graph_data['Player'] == 'Tony Parker')  | (graph_data['Player'] == 'Tony Allen') | (graph_data['Player'] == 'Derrick Rose') | (graph_data['Player'] == 'Russell Westbrook') 
| (graph_data['Player'] == 'Stephen Curry') | (graph_data['Player'] == 'Dwyane Wade') | (graph_data['Player'] == 'John Wall') | (graph_data['Player'] == 'Rajon Rondo')
| (graph_data['Player'] == 'Tracy McGrady') | (graph_data['Player'] == 'Dennis Schröder') | (graph_data['Player'] == 'Allen Iverson') | (graph_data['Player'] == 'Antoine Walker') 
| (graph_data['Player'] == 'Baron Davis') | (graph_data['Player'] == 'Andre Miller') | (graph_data['Player'] == 'Michael Jordan') | (graph_data['Player'] == 'Donovan Mitchell')
| (graph_data['Player'] == 'Vince Carter') | (graph_data['Player'] == 'Damian Lillard') | (graph_data['Player'] == 'Leandro Barbosa')
| (graph_data['Player'] == 'Gary Payton') | (graph_data['Player'] == 'Avery Johnson') | (graph_data['Player'] == 'Jeff Teague')
| (graph_data['Player'] == 'Kyrie Irving') | (graph_data['Player'] == 'Shandon Anderson') | (graph_data['Player'] == 'Norman Powell')
| (graph_data['Player'] == 'Monta Ellis') | (graph_data['Player'] == 'Kobe Bryant') | (graph_data['Player'] == 'Eric Bledsoe')
| (graph_data['Player'] == 'Avery Bradley') | (graph_data['Player'] == 'Jimmy Butler')  | (graph_data['Player'] == 'Bobby Jackson')
| (graph_data['Player'] == 'Richard Hamilton') | (graph_data['Player'] == 'Ronald Murray') | (graph_data['Player'] == 'Jameer Nelson')
| (graph_data['Player'] == 'Kerry Kittles') | (graph_data['Player'] == 'Jaylen Brown') | (graph_data['Player'] == 'Deron Williams')
| (graph_data['Player'] == 'James Harden') | (graph_data['Player'] == 'Lance Stephenson') | (graph_data['Player'] == 'Manu Ginóbili')
| (graph_data['Player'] == 'Goran Dragić')]

Tony = (graph_data['Player']=='Tony Parker')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Manu Ginóbili'),'#FF8200', graph_data.color)

size = 100 * ((graph_data['MP'] - graph_data['MP'].min()) / (graph_data['MP'].max() - graph_data['MP'].min()))
plt.figure(figsize=(17,9), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-3 FGA per 75", y="0-3 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-3 Feet Playoff Scoring (1997-2022) >1000 MP', xlabel='0-3 Feet FGA per 75', ylabel='0-3 Feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tony Parker', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Manu Ginóbili', markerfacecolor='#FF8200',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['0-3 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['0-3 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
graph_data = graph_data[(graph_data['Player'] != 'Tony Parker')]
graph_data = graph_data[(graph_data['Player'] != 'Manu Ginóbili')]
graph_data = graph_data[(graph_data['Player'] != 'Norman Powell')]
graph_data = graph_data[(graph_data['Player'] != 'Ronald Murray')]
label_point(graph_data['0-3 FGA per 75'], graph_data['0-3 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-3 Graph Tim
graph_data =  pd.read_csv('/content/nba_player_0-3_playoff_shooting_data.csv', encoding='utf8')
graph_data = graph_data[(graph_data['0-3 FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-3 FGA per 75", y="0-3 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-3 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-3 Feet FGA per 75', ylabel='0-3 Feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['0-3 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['0-3 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-3 FGA per 75'], graph_data['0-3 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-3 Graph Tim TS+
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')
graph_data = graph_data[(graph_data['0-3 FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-3 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-3 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-3 Feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='lower right')

# ridge regression
x_pred = graph_data['0-3 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-3 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['2P Jumpshot FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['2P Jumpshot PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])

graph_data = graph_data[(graph_data['2P Jumpshot FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="2P Jumpshot FGA per 75", y="2P Jumpshot PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='2P Jumpshot Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='2P Jumpshot FGA per 75', ylabel='2P Jumpshot PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['2P Jumpshot FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['2P Jumpshot PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['2P Jumpshot FGA per 75'], graph_data['2P Jumpshot PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-3P Graph TS+
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['2P Jumpshot FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75'] + graph_data['16-3P FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq'] + graph_data['16-3P Freq']
graph_data['2P Jumpshot PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS']) + (((graph_data['16-3P Freq']) / total_freq) * graph_data['16-3P feet PPS'])

graph_data = graph_data[(graph_data['2P Jumpshot FGA per 75'] >= 4)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]
graph_data = graph_data.sort_values('Player', ascending=False)

graph_data = graph_data[(graph_data['Player'] != 'Chris Paul')]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="2P Jumpshot FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='2P Jumpshot Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='2P Jumpshot FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['2P Jumpshot FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['2P Jumpshot FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'0-3_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 PPS Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['3-16 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq']
graph_data['3-16 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="3-16 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 feet FGA per 75', ylabel='3-16 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['3-16 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-16 FGA per 75'], graph_data['3-16 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 TS+ Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['3-16 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['10-16 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['10-16 Freq']
graph_data['3-16 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['10-16 Freq']) / total_freq) * graph_data['10-16 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-16 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-10 PPS Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-10 FGA per 75", y="3-10 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-10 FGA feet per 75', ylabel='3-10 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['3-10 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-10 FGA per 75'], graph_data['3-10 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'3-10_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-10 TS+ Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')


graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-10 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-10 feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['3-10 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'3-10_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-10 PPS Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['0-10 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['0-3 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['0-3 Freq']
graph_data['0-10 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['0-3 Freq']) / total_freq) * graph_data['0-3 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-10 FGA per 75", y="0-10 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-10 feet FGA per 75', ylabel='0-10 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['0-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['0-10 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-10 FGA per 75'], graph_data['0-10 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'0-10_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 0-10 TS+ Graph
graph_data =  pd.read_csv('/content/NBA_Playoff_all_shooting_data_df.csv', encoding='utf8')

graph_data['0-10 FGA per 75'] = graph_data['3-10 FGA per 75'] + graph_data['0-3 FGA per 75']
total_freq = graph_data['3-10 Freq'] + graph_data['0-3 Freq']
graph_data['0-10 feet PPS'] = (((graph_data['3-10 Freq']) / total_freq) * graph_data['3-10 feet PPS']) + (((graph_data['0-3 Freq']) / total_freq) * graph_data['0-3 feet PPS'])

graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="0-10 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='0-10 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='0-10 feet FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['0-10 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant')]
label_point(graph_data['0-10 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'0-10_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 TS+ Prime Graph
graph_data =  pd.read_csv('/content/Tim98_06_NBA_Playoff_Shortrange_df_1000mp_3shortfga.csv', encoding='utf8')
graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan 98 - 06')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant 06 - 12'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="TS+", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 FGA per 75', ylabel='TS+')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan 98 - 06', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant 06 - 12', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['TS+']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan 98 - 06')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant 06 - 12')]
label_point(graph_data['3-16 FGA per 75'], graph_data['TS+'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_Graph', bbox_inches='tight')
plt.show()

In [ ]:
#@title 3-16 PPS Prime Graph
graph_data =  pd.read_csv('/content/Tim98_06_NBA_Playoff_Shortrange_df_1000mp_3shortfga.csv', encoding='utf8')
graph_data = graph_data.sort_values('Player', ascending=False)
graph_data = graph_data[(graph_data['MP'] >= 2000)]
graph_data = graph_data[(graph_data['PP75'] >= 15)]

Tony = (graph_data['Player']=='Tim Duncan 98 - 06')
graph_data['color']= np.where(Tony==True , "#EF426F", "#00B2A9")
graph_data['color'] = np.where((graph_data.Player == 'Kobe Bryant 06 - 12'),'y', graph_data.color)

size = 100 * ((graph_data['PP75'] - graph_data['PP75'].min()) / (graph_data['PP75'].max() - graph_data['PP75'].min()))
plt.figure(figsize=(20,12), tight_layout=True)
sns.color_palette("flare", as_cmap=True)
ax = sns.regplot(data=graph_data, x="3-16 FGA per 75", y="3-16 feet PPS", fit_reg=False, scatter_kws={'facecolors':graph_data['color'], 'alpha':1.0, 's':size*4, 'linewidth':0})
ax.set(title='3-16 Feet Playoff Scoring (1997-2022) > 15 PP75 >2000 MP', xlabel='3-16 FGA per 75', ylabel='3-16 feet PPS')

legend_elements = [Line2D([0], [0], marker='o', color='#EAEAF2', label='Tim Duncan 98 - 06', markerfacecolor='#EF426F',  markersize=10),
                   Line2D([0], [0], marker='o', color='#EAEAF2', label='Kobe Bryant 06 - 12', markerfacecolor='y',  markersize=10)]
ax.legend(handles=legend_elements, loc='upper right')

# ridge regression
x_pred = graph_data['3-16 FGA per 75']
x_pred = np.array(x_pred).reshape(-1, 1)
y_pred = graph_data['3-16 feet PPS']
reg = linear_model.Ridge(alpha=10)
reg.fit(x_pred, y_pred)
pred = reg.predict(x_pred)
ax.plot(x_pred, pred, color="k", linewidth=3)

graph_data = graph_data[(graph_data['Player'] != 'Tim Duncan 98 - 06')]
graph_data = graph_data[(graph_data['Player'] != 'Kobe Bryant 06 - 12')]
label_point(graph_data['3-16 FGA per 75'], graph_data['3-16 feet PPS'], graph_data['Player'], plt.gca())
plt.savefig(f'3-16_Graph', bbox_inches='tight')
plt.show()